In [1]:
import pandas as pd
import pickle
import numpy as np
from collections import Counter

In [2]:
refined_dataset = pd.read_csv("../data/refined_dataset.csv")

In [8]:
refined_dataset.userId.unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [9]:
refined_dataset.movieId.unique()

array([  924,  6502,  3153, ..., 74541,  1780, 56274], dtype=int64)

In [4]:
refined_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           150001 non-null  int64  
 1   userId               150001 non-null  int64  
 2   title                150001 non-null  object 
 3   movieId              150001 non-null  int64  
 4   rating               150001 non-null  float64
 5   loc_clusters_users   150001 non-null  int64  
 6   loc_clusters_movies  150001 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 8.0+ MB


# Recommender via Movie

In [32]:
#load model
kmeans_movie = pickle.load(open('../models/kmeans_movie.pkl', 'rb'))

In [33]:
df = pd.DataFrame(np.array([[1, 4], [2,5], [994,5]]),
                   columns=['userId', 'rating'])

In [34]:
df

,userId,rating
0,1,4
1,2,5
2,994,5


In [35]:
#predict
cluster_movies = kmeans_movie.predict(df)

In [59]:
input_movie = input("Enter a movieID: ")
input_movie = int(input_movie)
cluster_movies = refined_dataset.loc[refined_dataset['movieId'] == input_movie, 'loc_clusters_movies']
print(cluster_movies.value_counts())
cluster_movies = Counter(cluster_movies).most_common(1)[0] # 4, 6 times
print(cluster_movies)
cluster_movies[0]

Enter a movieID:  994


4    8
7    7
5    6
3    4
9    3
6    3
2    3
1    3
8    2
0    2
Name: loc_clusters_movies, dtype: int64
(4, 8)


4

In [70]:
#what should the user see
movies = refined_dataset.loc[refined_dataset['loc_clusters_movies'] == cluster_movies[0], 'movieId']
movies
for c in range(5):
    if movies.iloc[c] == input:
        continue
    else:
       movie1 = refined_dataset.loc[refined_dataset['movieId'] == movies.iloc[c]]
       movie1 = movie1['title']
       print(movie1.iloc[1])

Adventures of Priscilla, Queen of the Desert, The (1994)
American Pie (1999)
And the Band Played On (1993)
Apollo 13 (1995)
As Good as It Gets (1997)


# Recommender via Movie

In [64]:
#load model
kmeans_user = pickle.load(open('../models/kmeans_user.pkl', 'rb'))

In [65]:
df = pd.DataFrame(np.array([[994, 3]]),
                   columns=['movieId', 'rating'])

In [66]:
cluster = kmeans_user.predict(df)

In [67]:
input_user = input("Enter a userid: ")
input_user = int(input_user)
cluster_users = refined_dataset.loc[refined_dataset['userId'] == input_user, 'loc_clusters_users']
print(cluster_users.value_counts())
cluster_users = Counter(cluster_users).most_common(1)[0] # 4, 6 times
print(cluster_users)
cluster_users[0]

Enter a userid:  994


9    110
3    100
0     87
2     23
5     20
6     11
8      5
1      2
Name: loc_clusters_users, dtype: int64
(9, 110)


9

In [68]:
users = refined_dataset.loc[refined_dataset['loc_clusters_users'] == cluster_users[0], 'userId']
users
list_movies = []
for c in range(1000):
    if users.iloc[c] == input_user:
        continue
    else:
       movie1 = refined_dataset.loc[refined_dataset['movieId'] == users.iloc[c]]
       movie1 = movie1['title']
       movie1 = movie1.iloc[1]
       list_movies.append(movie1)
uniqueElements = list(set(list_movies))
print(uniqueElements[0:5])

['American President, The (1995)', 'Powder (1995)', 'GoldenEye (1995)', 'Grumpier Old Men (1995)', 'Sudden Death (1995)']
